<a href="https://colab.research.google.com/github/anika64/ALL-Classification_ViTTiny/blob/main/VitTiny_SRA_ALL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install -q datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.8 MB/s eta 0:00:00


In [3]:
!pip uninstall -y transformers
!pip install transformers --upgrade --no-cache-dir


Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 107.3 MB/s eta 0:00:00


In [4]:
import transformers
print(transformers.__version__)


4.52.4


In [5]:
import zipfile, os

data_dir = '/content/data'
os.makedirs(data_dir, exist_ok=True)

fold_0_path = "/content/drive/MyDrive/fold_0.zip"
val_images_path = "/content/drive/MyDrive/val_images.zip"
csv_path = "/content/drive/MyDrive/C-NMC_test_prelim_phase_data_labels.csv"

with zipfile.ZipFile(fold_0_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(data_dir, 'train'))

with zipfile.ZipFile(val_images_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(data_dir, 'val'))


In [6]:
import pandas as pd
import glob

df = pd.read_csv(csv_path)

# Find matching image file path in either train or val folders
def find_image_path(filename, folder):
    matches = glob.glob(f"{folder}/**/{filename}", recursive=True)
    return matches[0] if matches else None

df["file_path"] = df["new_names"].apply(
    lambda name: find_image_path(name, os.path.join(data_dir, 'train')) or
                 find_image_path(name, os.path.join(data_dir, 'val'))
)

df = df.dropna(subset=["file_path"])
df = df.rename(columns={"labels": "label"})
df = df[["file_path", "label"]]  # Only keep necessary columns
df.head()


,file_path,label
0,/content/data/val/C-NMC_test_prelim_phase_data...,1
1,/content/data/val/C-NMC_test_prelim_phase_data...,1
2,/content/data/val/C-NMC_test_prelim_phase_data...,1
3,/content/data/val/C-NMC_test_prelim_phase_data...,0
4,/content/data/val/C-NMC_test_prelim_phase_data...,1


In [7]:
from datasets import Dataset

hf_dataset = Dataset.from_pandas(df)
hf_dataset = hf_dataset.train_test_split(test_size=0.2, seed=42)


In [8]:
from transformers import AutoImageProcessor

processor = AutoImageProcessor.from_pretrained("WinKawaks/vit-tiny-patch16-224")


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [9]:
from PIL import Image

def preprocess(example):
    image = Image.open(example["file_path"]).convert("RGB")
    processed = processor(images=image, return_tensors="pt")
    return {
        "pixel_values": processed["pixel_values"].squeeze(),
        "label": int(example["label"])
    }

hf_dataset = hf_dataset.map(preprocess, remove_columns=hf_dataset["train"].column_names, batched=False)


Map:   0%|          | 0/1493 [00:00<?, ? examples/s]

Map:   0%|          | 0/374 [00:00<?, ? examples/s]

In [10]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained(
    "WinKawaks/vit-tiny-patch16-224",
    num_labels=2,
    ignore_mismatched_sizes=True
)


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([2, 192]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,
    remove_unused_columns=False
)


In [20]:
!pip install -q scikit-learn


In [21]:
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}


In [23]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(labels, preds)}

# Recreate Trainer
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=hf_dataset["test"],
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate()
print(f"Test Accuracy: {metrics['eval_accuracy']:.4f}")


Test Accuracy: 0.8904


In [16]:
save_path = "/content/drive/MyDrive/vit-tiny-leukemia"
model.save_pretrained(save_path)
processor.save_pretrained(save_path)


['/content/drive/MyDrive/vit-tiny-leukemia/preprocessor_config.json']

In [25]:
from sklearn.metrics import classification_report
import torch
import numpy as np

model.eval()

all_preds = []
all_labels = []

for example in hf_dataset["test"]:
    # Convert list to tensor and add batch dimension
    pixel_values = torch.tensor(example["pixel_values"]).unsqueeze(0)
    inputs = {"pixel_values": pixel_values}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        pred = torch.argmax(logits, dim=1).item()

    all_preds.append(pred)
    all_labels.append(example["label"])


In [26]:
from sklearn.metrics import classification_report

target_names = ["HEM", "ALL"]  # Based on label 0 = HEM, 1 = ALL
print(classification_report(all_labels, all_preds, target_names=target_names, digits=2))


              precision    recall  f1-score   support

         HEM       0.80      0.89      0.84       124
         ALL       0.94      0.89      0.92       250

    accuracy                           0.89       374
   macro avg       0.87      0.89      0.88       374
weighted avg       0.90      0.89      0.89       374

